In [2]:
import torch
import os
import pickle
import numpy as np
import itertools
import librosa
import random
from random import shuffle
import codecs
import errno
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision.datasets.mnist
from torchvision import transforms
from tqdm import tqdm
import math

In [3]:
print(os.getcwd())

G:\Git\Deep Learning\Assignment4


In [4]:
file = open("hw4_trs.pkl",'rb')
train_data = pickle.load(file)
file.close()

file = open("hw4_tes.pkl",'rb')
test_data = pickle.load(file)
file.close()

In [5]:
train_data.shape
training = []

In [6]:
training = []
for sig in range(500):
    stft = librosa.stft(train_data[sig], n_fft=1024, hop_length=512)
#     stft.resize((513,32))
    training.append(np.abs(stft).T)

In [8]:
training[0].shape

(32, 513)

In [7]:
test_data.shape

(200, 22631)

In [8]:
test_data

array([[ 0.00127268, -0.00018181,  0.00027272, ..., -0.00363621,
        -0.0034544 , -0.00418165],
       [-0.00023365,  0.00023365,  0.        , ..., -0.00700938,
        -0.00763243, -0.00957948],
       [ 0.0014514 , -0.00019352,  0.00029028, ..., -0.00232224,
        -0.00299957, -0.00261252],
       ...,
       [ 0.00022017,  0.00036695,  0.00014678, ...,  0.02803493,
         0.01812729,  0.00910034],
       [ 0.00011644,  0.00034932,  0.00034932, ..., -0.06386784,
        -0.08779645, -0.10881403],
       [ 0.00034393,  0.00080251,  0.00074519, ..., -0.00045858,
        -0.00424185, -0.00492971]], dtype=float32)

In [9]:
testing = []
for sig2 in range(200):
    stft = librosa.stft(test_data[sig2], n_fft=1024, hop_length=512)
#     print(stft.shape)
#     stft.resize((513,45))
    training.append(np.abs(stft).T)

In [10]:
np.append(np.arange(3,10), np.arange(11,20))

array([ 3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [11]:
train_data[0:9,:].shape

(9, 16180)

In [12]:
positive_list = []
negative_list = []
m = [1,2,3]
print(shuffle(m))

None


In [13]:
def get_pairs(number_of_speakers):
    positive_pairs = np.zeros((number_of_speakers,9,2))
    negative_pairs = np.zeros((number_of_speakers,9,2))
    person = np.zeros((number_of_speakers,9))
    for i in range(number_of_speakers):
#         print(i)
        complete_set = set(range(number_of_speakers * 10))
        positive_set = set(range(i, i+10))
        negative_set = set(complete_set - positive_set)

        # print(list(positive_set))
        positive_list = list(positive_set)
        # print(len(positive_list))
        negative_list = list(negative_set)
#         print(positive_list)
#         print(negative_list)
#         shuffle(positive_list)
#         shuffle(negative_list)

        positive_combinations = np.array(random.sample(list(itertools.combinations(positive_list, 2)), 9) )
        negative_combinations = np.array(random.sample(list(itertools.product(positive_list, negative_list)), 9))
#         print(positive_combinations)
#         print(negative_combinations)
        positive_pairs[i] = positive_combinations
        negative_pairs[i] = negative_combinations
        person[i] = i
    return positive_pairs, negative_pairs, person

In [14]:
positive_pairs, negative_pairs, person = get_pairs(50)

In [15]:
print(positive_pairs)

[[[ 3.  9.]
  [ 0.  2.]
  [ 0.  6.]
  [ 2.  5.]
  [ 5.  6.]
  [ 1.  2.]
  [ 4.  6.]
  [ 2.  6.]
  [ 3.  5.]]

 [[ 1.  5.]
  [ 4.  7.]
  [ 7.  9.]
  [ 3.  4.]
  [ 9. 10.]
  [ 2.  9.]
  [ 5.  6.]
  [ 4.  8.]
  [ 7.  8.]]

 [[ 7. 11.]
  [ 3.  8.]
  [ 3.  5.]
  [ 9. 11.]
  [ 2.  3.]
  [ 3.  9.]
  [ 5.  7.]
  [ 2.  8.]
  [ 2.  6.]]

 [[ 7.  9.]
  [ 4. 10.]
  [ 5. 11.]
  [ 6. 12.]
  [ 7. 11.]
  [ 4.  8.]
  [ 3.  7.]
  [ 5. 12.]
  [ 4.  7.]]

 [[ 7. 11.]
  [ 7. 10.]
  [ 5.  8.]
  [ 5. 12.]
  [ 6.  9.]
  [ 6. 10.]
  [ 8. 10.]
  [ 7. 12.]
  [11. 13.]]

 [[ 9. 10.]
  [ 9. 14.]
  [12. 13.]
  [ 8.  9.]
  [ 8. 10.]
  [ 9. 12.]
  [12. 14.]
  [ 8. 11.]
  [ 5. 10.]]

 [[ 9. 11.]
  [ 7. 14.]
  [ 8. 12.]
  [ 8. 13.]
  [ 9. 10.]
  [12. 15.]
  [ 6. 11.]
  [ 7. 10.]
  [ 6.  8.]]

 [[ 8. 11.]
  [ 9. 12.]
  [10. 14.]
  [13. 15.]
  [11. 14.]
  [ 7. 14.]
  [12. 14.]
  [ 7. 12.]
  [ 9. 10.]]

 [[ 9. 15.]
  [ 8. 17.]
  [ 9. 10.]
  [ 9. 16.]
  [11. 12.]
  [10. 17.]
  [13. 15.]
  [11. 14.]
  [13. 16.]]

 [[12. 17.

In [16]:
positive_pairs_reshaped = positive_pairs.reshape((50 * 9), 2)
negative_pairs_reshaped = negative_pairs.reshape((50 * 9), 2)
person = person.reshape((450, 1))

In [17]:
# for j in range(len(positive_pairs)):
# #     j will be from 0 to 50
#     positive_list = []
#     negative_list = []
#     for k in range(len(positive_pairs)):
        
#         pos_index1  =  positive_pairs[k][0]
#         pos_index2  =  positive_pairs[k][1]
#         print(pos_index2)
#         neg_index1  =  negative_pairs[k][0]
#         neg_index2  =  negative_pairs[k][1]
        
#         pos_pair = (train_data[pos_index1, :], train_data[pos_index2, :], math.floor(pos_index1 / 50))
#         neg_pair = (train_data[neg_index1, :], train_data[neg_index2, :], math.floor(pos_index1 / 50))
        
        

In [18]:
# ref: https://becominghuman.ai/siamese-networks-algorithm-applications-and-pytorch-implementation-4ffa3304c18


# class LSTM(nn.Module):
    
#     def __init__(self, input_size, hidden_size):
#         super(LSTM, self).__init__()
#         self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
#         self.linear = nn.Linear(hidden_size, 513)
#         self.act = torch.sigmoid
      
#     def forward(self, data):
#         res = []
#         for i in range(2): # Siamese nets; sharing weights
#             x = data[i]
#             print('data')
#             print(self.rnn)
#             pred, hidden = self.rnn(x[0])
#             pred = self.act(self.linear(pred)).view(pred.data.shape[0], BATCH, 513)
#             res.append(pred)
         
#         res = torch.abs(res[1] - res[0])
#         res = self.linear2(res)
#         return res

In [19]:
train_positive_pair_loader = torch.utils.data.DataLoader(positive_pairs_reshaped, batch_size=1)
train_negative_pair_loader = torch.utils.data.DataLoader(negative_pairs_reshaped, batch_size=1)
train_label_loader = torch.utils.data.DataLoader(person, batch_size=1)

In [20]:
# https://hackernoon.com/one-shot-learning-with-siamese-networks-in-pytorch-8ddaab10340e
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

In [21]:
# https://hackernoon.com/one-shot-learning-with-siamese-networks-in-pytorch-8ddaab10340e
# ref: https://becominghuman.ai/siamese-networks-algorithm-applications-and-pytorch-implementation-4ffa3304c18
class LSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.rnn = nn.GRU(input_size, hidden_size, 1)
        self.linear1 = nn.Linear(64, 32)
        self.linear2 = nn.Linear(32, 16)
        
        self.act = F.relu
        
    def forward(self, data):
#         pred, hidden = self.rnn(x, None)
#         pred = self.act(self.linear(pred)).view(pred.data.shape[0], BATCH, 513)
#         res = []
#         for i in range(3): # Siamese nets; sharing weights
        x = torch.tensor(data).cuda()
        pred, hidden = self.rnn(x[np.newaxis, :, :])
        pred = pred.view(pred.shape[0], -1)
        pred = self.act(self.linear1(pred))
        pred = self.linear2(pred)
        return pred
#         print(res[0], res[1])
#         res = torch.dot(torch.squeeze(res[1]), torch.squeeze(res[0]))
#         res = F.sigmoid(res)
#         return res

neural_network = LSTM(513,2)
neural_network = neural_network.cuda()
# loss_function= nn.TripletMarginLoss()
loss_function= ContrastiveLoss()
para = neural_network.parameters()
optimizer = torch.optim.Adam(params=para, lr=0.00001)
loss_list = []
epochs = 100
# count = 0
for i in range(epochs):
    positive_pairs = iter(train_positive_pair_loader)
    negative_pairs = iter(train_negative_pair_loader)
    labels = iter(train_label_loader)
    while True:
        try:
#             count = count + 1
            positive_sets = positive_pairs.next()[0]
            negative_sets = negative_pairs.next()[0]

            target = labels.next()[0]
#             print(target)
            anchor = training[int(positive_sets[0].numpy())]
            a = training[int(positive_sets[1].numpy())]
            b = training[int(negative_sets[1].numpy())]
#             positive_tensors = [training[int(positive_sets[0].numpy())],training[int(positive_sets[1].numpy())]]
#             negative_tensors = [training[int(positive_sets[0].numpy())],training[int(negative_sets[1].numpy())]]
#             print(positive_tensors[0].shape)
            optimizer.zero_grad()
            for i in range(2):
                if i == 0:
                    embed_anchor = neural_network(anchor)
                    embed_a  = neural_network(a)
                    loss = loss_function(embed_anchor ,embed_a, 1 )
                else:
                    embed_anchor = neural_network(anchor)
                    embed_b  = neural_network(b)
                    loss = loss_function(embed_anchor ,embed_b, 0 )
#             embed_anchor = neural_network(anchor)
#             embed_a  = neural_network(a)
#             embed_b  = neural_network(b)
            
#             negative_set_output  = neural_network(negative_tensors)
#             print(positive_set_output)
#             print(negative_set_output)
#             loss = loss_function(embed_anchor ,embed_a, embed_b )
#             loss_positive = F.cross_entropy(positive_set_output , torch.tensor(1).type(torch.LongTensor).cuda())
#             loss_negative = F.cross_entropy(negative_set_output , torch.tensor(0).type(torch.LongTensor).cuda())
#             print('positive loass')
#             print(loss_positive)
#             print('negative loss')
#             print(loss_negative)
#             loss = loss_positive + loss_negative
            loss.backward()
            optimizer.step()
        except StopIteration:
            break
    print(loss)
    loss_list.append(loss.data.cpu().numpy())
print(loss.data)
plt.plot(range(epochs), loss_list)
plt.show() 

tensor(0.1372, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0376, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0048, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0007, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.6000e-11, device='cuda:0', grad_fn=

KeyboardInterrupt: 

In [ ]:
positive_pairs, negative_pairs, person = get_pairs(20)

In [ ]:
positive_pairs.shape

In [ ]:
positive_pairs_reshaped = positive_pairs.reshape((20 * 9), 2)
negative_pairs_reshaped = negative_pairs.reshape((20 * 9), 2)
person = person.reshape((180, 1))

In [ ]:
test_positive_pair_loader = torch.utils.data.DataLoader(positive_pairs_reshaped, batch_size=1)
test_negative_pair_loader = torch.utils.data.DataLoader(negative_pairs_reshaped, batch_size=1)
test_label_loader = torch.utils.data.DataLoader(person, batch_size=1)

In [ ]:
len(test_negative_pair_loader)

In [ ]:
with torch.no_grad():
    positive_pairs = iter(test_positive_pair_loader)
    negative_pairs = iter(test_negative_pair_loader)
    positive = 0
    negative = 0
    dist1 = np.zeros(180)
    dist2 = np.zeros(180)
    i = 0
#     labels = iter(train_label_loader)
    while True:
        try:
#             count = count + 1
            positive_sets = positive_pairs.next()[0]
            negative_sets = negative_pairs.next()[0]
            
#             target = labels.next()[0]
#             print(target)
            anchor = training[int(positive_sets[0].numpy())]
            a = training[int(positive_sets[1].numpy())]
            b = training[int(negative_sets[1].numpy())]
#             print(positive_sets[0],positive_sets[1])
#             print(positive_sets[0], negative_sets[1])
#             positive_tensors = [training[int(positive_sets[0].numpy())],training[int(positive_sets[1].numpy())]]
#             negative_tensors = [training[int(positive_sets[0].numpy())],training[int(negative_sets[1].numpy())]]
#             print(positive_tensors[0].shape)

#             embed_anchor = neural_network(anchor)
#             embed_a  = neural_network(a)
#             embed_b  = neural_network(b)
            
            output1 = neural_network(anchor)
            output2 = neural_network(a)
            output3 = neural_network(b)
            euclidean_distance1 = F.pairwise_distance(torch.tensor(output1), torch.tensor(output2))
            euclidean_distance2 = F.pairwise_distance(torch.tensor(output1), torch.tensor(output3))
            dist1[i] = euclidean_distance1
            dist2[i] = euclidean_distance2
            
            i = i+1
#             threshold = 0.31
#             if euclidean_distance1 < threshold:
#                 positive = positive + 1
#             if euclidean_distance2 >= threshold:
#                 negative = negative + 1
#             print('dist1', euclidean_distance1)
#             print('dist2', euclidean_distance2)
        except StopIteration:
            a = np.sum(dist2 - dist1)
            print('metric' , a)
            print(positive)
            print(negative)
            break 
            
#             negative_set_output  = neural_network(negative_tensors)
#             print(positive_set_output)
#             print(negative_set_output)


In [11]:
x = np.array([0,1,2,3,4])
y = x[0:3]
print(y)

[0 1 2]
